__NAME:__ __FULLNAME__  
__SECTION:__ __NUMBER__  
__CS 5703: Machine Learning Practices__

# Homework 9: Decision Tree Classifiers

## Assignment Overview
Follow the TODOs and read through and understand any provided code. 
Post any questions you might have to the class Slack. 
For all plots, make sure all necessary axes and curves are clearly and 
accurately labeled. Include figure/plot titles appropriately as well.


### Task
For this assignment you will be exploring Decision Tree Classifiers to 
predict potentially fraudulent providers from summary statistics of their filed healthcare claims. 

These data were re-configured from a dataset collected for the purpose of detecting Health care Provider Fraud. Total Medicare spending increases exponentially due to fraud in Medicare claims. Healthcare fraud involves health care providers, physicians, patients, and beneficiaries acting in tandum to construct fraudulent claims.


__Features__  
The features are aggregate statistics computed as either the mean or the sum.
For the following features, the column represents the average value for the provider's claims:  
* InscClaimAmtReimbursed  
* DeductibleAmtPaid
* NoOfMonths_PartACov
* NoOfMonths_PartBCov
* IPAnnualReimbursementAmt
* IPAnnualDeductibleAmt
* OPAnnualReimbursementAmt
* OPAnnualDeductibleAmt
* NumPhysiciansSeen
* NumProcedures
* NumDiagnosisClaims
* Age
 
For the following features, the column represents the total number among the provider's claims:  
* ChronicCond_Alzheimer  
* ChronicCond_Heartfailure  
* ChronicCond_KidneyDisease  
* ChronicCond_Cancer  
* ChronicCond_ObstrPulmonary  
* ChronicCond_Depression  
* ChronicCond_Diabetes  
* ChronicCond_IschemicHeart  
* ChronicCond_Osteoporasis  
* ChronicCond_rheumatoidarthritis  
* ChronicCond_stroke  
* RenalDiseaseIndicator  

These data were amalagmated from the [HEALTHCARE PROVIDER FRAUD DETECTION ANALYSIS](https://www.kaggle.com/rohitrox/healthcare-provider-fraud-detection-analysis) data set on Kaggle.


### Objectives
* Introduction to Decision Trees


### General References
* [Guide to Jupyter](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook)
* [Python Built-in Functions](https://docs.python.org/3/library/functions.html)
* [Python Data Structures](https://docs.python.org/3/tutorial/datastructures.html)
* [Numpy Reference](https://docs.scipy.org/doc/numpy/reference/index.html)
* [Numpy Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)
* [Summary of matplotlib](https://matplotlib.org/3.1.1/api/pyplot_summary.html)
* [DataCamp: Matplotlib](https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python?utm_source=adwords_ppc&utm_campaignid=1565261270&utm_adgroupid=67750485268&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=332661264365&utm_targetid=aud-299261629574:dsa-473406587955&utm_loc_interest_ms=&utm_loc_physical_ms=9026223&gclid=CjwKCAjw_uDsBRAMEiwAaFiHa8xhgCsO9wVcuZPGjAyVGTitb_-fxYtkBLkQ4E_GjSCZFVCqYCGkphoCjucQAvD_BwE)
* [Pandas DataFrames](https://urldefense.proofpoint.com/v2/url?u=https-3A__pandas.pydata.org_pandas-2Ddocs_stable_reference_api_pandas.DataFrame.html&d=DwMD-g&c=qKdtBuuu6dQK9MsRUVJ2DPXW6oayO8fu4TfEHS8sGNk&r=9ngmsG8rSmDSS-O0b_V0gP-nN_33Vr52qbY3KXuDY5k&m=mcOOc8D0knaNNmmnTEo_F_WmT4j6_nUSL_yoPmGlLWQ&s=h7hQjqucR7tZyfZXxnoy3iitIr32YlrqiFyPATkW3lw&e=)
* [Sci-kit Learn Linear Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Sci-kit Learn Ensemble Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble)
* [Sci-kit Learn Metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
* [Sci-kit Learn Model Selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)
* [Sci-kit Learn Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
* [Sci-kit Learn Preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)  
* [Decision Trees](https://medium.com/machine-learning-101/chapter-3-decision-trees-theory-e7398adac567)

### Hand-In Procedure
* Execute all cells so they are showing correct results
* Notebook (from Jupyter or Colab):
  + Submit this file (.ipynb) to the Gradescope Notebook HW9 dropbox
* Note: there is no need to submit a PDF file or to submit directly to Canvas

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re, os, pathlib
import time as timelib

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.metrics import log_loss, f1_score, precision_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
import joblib
import pickle as pkl

# Default figure parameters
plt.rcParams['figure.figsize'] = (6,5)
plt.rcParams['font.size'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['figure.constrained_layout.use'] = False
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12


plt.style.use('ggplot')

In [ ]:
# COLAB ONLY
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# COLAB ONLY
#
# THIS IMPORTS 3 CUSTOM .py FILES 
# 
# These are the same python files as we used in HW08
#
# If you are running this on a local machine, don't execute this cell
#

# this is a little weird colab doesn't play _super_ nice with local 
# python files
# note that this is not programming best practice
exec(open(
    '/content/drive/My Drive/Colab Notebooks/visualize.py', 'r'
).read())
exec(open(
    '/content/drive/My Drive/Colab Notebooks/metrics_plots.py', 'r'
).read())
exec(open(
    '/content/drive/My Drive/Colab Notebooks/pipeline_components.py', 'r'
).read())

In [ ]:
# for local runtimes only (e.g., Jupyter)
from visualize import *
from metrics_plots import *
from pipeline_components import *

# LOAD DATA

In [ ]:
# TODO: set path appropriately. 
fname = "/content/drive/My Drive/MLP_2021/datasets/health_provider_fraud.csv"
#fname = "health_provider_fraud.csv"
claims_data = pd.read_csv(fname)
claims_data.shape

In [ ]:
""" PROVIDED
Display data info
"""
claims_data.info()

In [ ]:
""" PROVIDED
Display the head of the data
"""
claims_data.head()

In [ ]:
""" PROVIDED
Display the summary statistics
Make sure you skim this
"""
claims_data.describe()

# PRE-PROCESS DATA

In [ ]:
""" PROVIDED
Construct preprocessing pipeline
"""
selected_features = claims_data.columns.drop(['Provider'])
scaled_features = ['InscClaimAmtReimbursed', 'DeductibleAmtPaid',
                   'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
                   'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt']

pipe = Pipeline([
    ('RowDropper', DataSampleDropper()),
    ('FeatureSelector', DataFrameSelector(selected_features)),
    ('Scale', DataScaler(scaled_features))
])


In [ ]:
""" Provided: execute cell
Pre-process the data using the defined pipeline
"""
processed_data = pipe.fit_transform(claims_data)
processed_data.shape

In [ ]:
""" PROVIDED: execute cell
Verify all NaNs removed
"""
processed_data.isna().sum()

# VISUALIZE DATA

In [ ]:
""" PROVIDED
Plot the class distributions for no potential fraud and potential fraud
"""
class_counts = pd.value_counts(processed_data['PotentialFraud'])
class_counts.plot(kind='bar', rot=0, figsize=(10,3))
plt.title("Potential Cases of Fraud")
plt.ylabel("Count")

# Display the class fractions
nsamples, nfeatures = processed_data.shape
class_counts / nsamples

In [ ]:
""" PROVIDED
Extract indices of the postive and negative cases
"""
pos = processed_data['PotentialFraud'] == 1
neg = processed_data['PotentialFraud'] == 0

# Decision Tree Classifiers

### Model Exploration

In [ ]:
""" PROVIDED
Split data into X (the inputs) and y (the outputs)

Hold out a subset of the data, before training and cross validation
using train_test_split, with stratify equal to something other than NONE, 
and a test_size fraction of .2.

For this exploratory section, the held out set of data is a validation set.
For the GridSearch section, the held out set of data is a test set.
"""
targetnames = ['NonFraud', 'Fraud']

# Create the inputs and outputs
X = processed_data.drop(['PotentialFraud'], axis=1).copy()
y = processed_data['PotentialFraud'].values.ravel()

# Split data into train and test sets
Xtrain, Xval, ytrain, yval = train_test_split(X, y, stratify=y, random_state=1138, test_size=0.5)
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

In [ ]:
""" TODO
Explore interesting hyper-parameters. Train multiple decision trees using the training set only.
Pick your favorite model to leave within your submitted report.
"""
# TODO: Create and fit the model
tree_model = DecisionTreeClassifier(???)
tree_model.fit(???)

In [ ]:
# PROVIDED: Predict with the model on the validation set
preds_val = tree_model.predict(Xval)

# Obtain prediction probabilities for the test set, using 
proba_val = tree_model.predict_proba(Xval)  

# Obtain the classifier accuracy score for the test set using the
scores = tree_model.score(Xval, yval)

scores

In [ ]:
""" PROVIDED
Display the confusion matrix, KS plot, ROC curve, and PR curve for the validation set
using metrics_plots.ks_roc_prc_plot

The red dashed line in the PRC is indicative of the expected performance for a random
classifier, which would predict postives at the rate of occurance within the data set
"""
# Confusion Matrix
cmtx_val = confusion_matrix(yval, preds_val)
confusion_mtx_colormap(cmtx_val, targetnames, targetnames)

# Curves
# Note, you'll want the probability class predictions for the class label 1
# See the API page for the DecisionTreeClassifier predict_proba; proba_val[:,1]
roc_prc_results_val  = ks_roc_prc_plot(yval, proba_val[:,1])

# Obtain the PSS and F1 Score
pss_val = skillScore(yval, preds_val)

# pss_val = metrics_plots.skillScore(ytest, preds_val)
f1_val = f1_score(yval, preds_val)
print("PSS: %.4f" % pss_val[0])
print("F1 Score %.4f" % f1_val)

In [ ]:
""" PROVIDED
Export the image of the tree model
"""
from IPython.display import Image
export_graphviz(tree_model, out_file='exploratory_model.dot', 
                feature_names=X.columns, class_names=targetnames, 
                rounded=True, filled=True)
!dot -Tpng exploratory_model.dot > e_model.png
Image(filename='e_model.png')

# GRID SEARCH CV

In [ ]:
def remove_duplicates(arr):
    ''' 
    Remove duplicates from an array
    '''
    out = []
    for i in arr:
        if not i in out:
            out.append(i)
    return out

In [ ]:
""" TODO
Set up and run the grid search using GridSearchCV and the following 
settings:
* The below scoring dictionary for scoring
* refit set to 'f1' as the optimized metric
* Choose a range of regularization types and parameters
"""
# Optimized metric
opt_metric = 'f1'
scoring = {opt_metric:opt_metric}

# Flag to re-load previous run regardless of whether the file exists
#force = False
force = True

# File previous run is saved to
srchfname = "/content/drive/My Drive/Colab Notebooks/hw9_search_sol_" + opt_metric + ".pkl"
#srchfname = "hw9_search_sol_" + opt_metric + ".pkl"


# SETUP EXPERIMENT HYPERPARAMETERS
# TODO

# TODO: Create the dictionary of hyper-parameters to try
hyperparams = {# TODO}


# RUN EXPERIMENT
time0 = timelib.time()
search = None
if force or (not os.path.exists(srchfname)):
    # Create the GridSearchCV object
    base_model = DecisionTreeClassifier()
    search = GridSearchCV(base_model, hyperparams, scoring=scoring, refit=opt_metric,
                          cv=40, n_jobs=-1, verbose=2, return_train_score=True)
    
    # TODO: Execute the grid search by calling fit using the training data
    search.fit(???)
    
    # Save the grid search object
    joblib.dump(search, srchfname)
    print("Saved %s" % srchfname)
else:
    # TODO: Re-load the grid search object
    search = joblib.load(srchfname)
    print("Loaded %s" % srchfname)

time1 = timelib.time()
duration = time1 - time0
print("Elapsed Time: %.2f min" % (duration / 60))

search

# RESULTS

In [ ]:
""" PROVIDED
Display the head of the results for the grid search
See the cv_results_ attribute
"""
all_results = search.cv_results_
df_res = pd.DataFrame(all_results)
df_res.head(3)

In [ ]:
""" PROVIDE
Obtain the best model from the grid search and 
fit it to the full training data
"""
best_model = search.best_estimator_
best_model.fit(Xtrain, ytrain)

In [ ]:
""" PROVIDED
Export the image of the best model
use export_graphviz
"""
export_graphviz(best_model, out_file='best_model.dot', 
                feature_names=X.columns, class_names=targetnames,
                rounded=True, filled=True)
!dot -Tpng best_model.dot > b_model.png
Image(filename='b_model.png')

In [ ]:
""" PROVIDED
Plot a histogram of the val scores from the best model.
Compare the distribution of probabilities for positive and negative examples
using boxplots.

Create one subplot of the distribution of all the probabilities, with a histogram. 
Create a second subplot comparing the distribution of the scores of the 
positive examples with the distribution of the negative examples, with boxplots.
"""
# Obtain the pos and neg indices
pos_inds = yval == 1
neg_inds = yval == 0

# Obtain prediction probabilities for the test set (use model.predict_proba)
proba_val = best_model.predict_proba(Xval)

# Separate the probabilities for the pos and neg examples
proba_pos = proba_val[pos_inds, 1]
proba_neg = proba_val[neg_inds, 1]

# Plot the distribution of all probabilities
nbins = 21
plt.figure(figsize=(15,3))
plt.subplot(1,3,1)
plt.hist(proba_val[:,1], bins=nbins)
plt.xlabel('probability')
plt.ylabel('count')
plt.title("Distribution of Instance Probabilities")

plt.subplot(1,3,2)
plt.hist(proba_neg, bins=nbins, alpha=.5)
plt.hist(proba_pos, bins=nbins, alpha=.5)
plt.xlabel('probability')
plt.ylabel('count')
plt.title("Distribution of Probabilities by Class")
plt.legend(['neg', 'pos'])

# Plot the boxplots of the pos and neg examples
plt.subplot(1,3,3)
boxplot = plt.boxplot([proba_neg, proba_pos], patch_artist=True, sym='.')
boxplot['boxes'][0].set_facecolor('pink')
boxplot['boxes'][1].set_facecolor('lightblue')
plt.xticks(ticks=[1, 2], labels=['-', '+'])
plt.xlabel("class")
plt.ylabel("probability")
plt.title("Probabilities non-Fraud(-) v. Fraud(+)")

## Compare Benchmark to GridSearchCV Best Model

In [ ]:
tree_model

In [ ]:
best_model

In [ ]:
# PROVIDED

# Predict with the benchmark model on the validation set
preds_val_bench = tree_model.predict(Xval)

# Predict with the best model on the test set
preds_val_best = best_model.predict(Xval)

# Obtain prediction probabilities for the benchmark model on val set
proba_val_bench = tree_model.predict_proba(Xval)

# Obtain prediction probabilities for the best model on test set
proba_val_best = best_model.predict_proba(Xval)

In [ ]:
# PROVIDED

# Benchmark tree model validation set confusion matrix
cmtx_val_bench = confusion_matrix(yval, preds_val_bench)
confusion_mtx_colormap(cmtx_val_bench, targetnames, targetnames)

# Best tree model test set confusion matrix
cmtx_val_best = confusion_matrix(yval, preds_val_best)
confusion_mtx_colormap(cmtx_val_best, targetnames, targetnames)

In [ ]:
# PROVIDED
# Curves (i.e. ROC, PRC, etc) use metrics_plots.ks_roc_prc_plot and the 
# the probabilities for the class label of 1

# Benchmark tree validation set performance
roc_prc_results_val_bench  = ks_roc_prc_plot(yval, proba_val_bench[:,1])

# Best tree model validation set performance
roc_prc_results_val_best  = ks_roc_prc_plot(yval, proba_val_best[:,1])

# Discussion

1. Discuss the difference in AUC between your hand-selected model and the best model found by GridSearch

__TODO__

2. How many different hyper-parameter sets did GridSearch consider?

__TODO__

3. What was the best set of hyper-parameters according to the GridSearch?

__TODO__

4. Discuss the difference in PR-AUC between your hand-selected model and the best model found by GridSearch

__TODO__

5. Examining the learned trees for both models, which features appear to be most important in performing this
classification task?

__TODO__

6. Relative to the validation data set, what is the best probability threshold to use to distinguish positive from negative classes?  Why?

